In [1]:
import sys
import os
# 获取套件的路径
package_path = os.path.join(os.getcwd(), '/Users/chieray/TMBA-ATD-coding/因子研究/python')  # 替换为你套件的实际路径
sys.path.append('/Users/chieray/TMBA-ATD-coding/因子研究/python')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re  # 添加这行
from scipy.cluster import hierarchy as sch
from scipy.spatial.distance import squareform
from scipy import stats
import statsmodels.api as sm
from scipy.optimize import minimize
import foctorvaluation
import factor_merger
import get_data
import factor_generator

/Users/chieray/TMBA-ATD-coding/因子研究/python/get_data.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[['symbol', 'name']] = df['symbol'].str.split(n=1, expand=True)


In [2]:
sys.path.append('/Users/chieray/Library/CloudStorage/GoogleDrive-smcjerry8@gmail.com/我的雲端硬碟/mbq')
from TQuantLab_Data.scripts.load_data import DataManager
manager = DataManager()
manager.list_datasets()

可用的資料集:
- 證券屬性資料表
- 月營收
- 股利政策
- 資本形成
- 集保庫存
- 三大法人_融資券_當沖
- 股價交易資訊
- 股票日交易註記資訊
- 交易日期表
- 會計師簽證財務資料
- 公司自結數
- 合併收購
- 董事長與高階主管變動事件
- 全面改選統計
- 董監全體持股狀況
- 庫藏股實施事件簿
- 董監申報轉讓_未轉讓
- 董監申報轉讓_轉讓
- 私募應募人與公司的關係
- 月營收_法人機構專屬版本


In [3]:
def resample_to_period(df, period):
    df['datetime'] = df.index
    df = df.groupby(df.index.to_period(period)).apply(lambda x: x.iloc[-1])
    df.index = df['datetime']
    df = df.drop(columns=['datetime'])
    return df

In [4]:
period='W'

In [5]:
調整係數=manager.tool_api_data("調整係數")
adjopen=industry=manager.tool_api_data("開盤價")['Open']*調整係數['Adjust_Factor']
adjhigh=industry=manager.tool_api_data("最高價")['High']*調整係數['Adjust_Factor']
adjlow=industry=manager.tool_api_data("最低價")['Low']*調整係數['Adjust_Factor']
adjclose=industry=manager.tool_api_data("收盤價")['Close']*調整係數['Adjust_Factor']
adjprice=(adjopen+adjhigh+adjlow+adjclose)/4
adjprice=adjprice.shift(-1)
monthadjprice=resample_to_period(adjprice, period=period)
exp_monthreturn=(monthadjprice.shift(-1)/monthadjprice)-1
monthreturn=exp_monthreturn.shift(2)

資料集 '調整係數' 包含其他欄位（['Adjust_Factor']），進行 pivot 操作。
資料集 '開盤價' 包含其他欄位（['Open']），進行 pivot 操作。
資料集 '最高價' 包含其他欄位（['High']），進行 pivot 操作。
資料集 '最低價' 包含其他欄位（['Low']），進行 pivot 操作。
資料集 '收盤價' 包含其他欄位（['Close']），進行 pivot 操作。


In [10]:
market=manager.get_data('股價交易資訊',common_stock = False)
market=market[market['證券名稱']=='IX0001']
market['年月'] = pd.to_datetime(market['資料日'], format='%Y/%m/%d')
market.rename(columns= {'年月': 'datetime'}, inplace=True)
market=market.set_index('datetime')
###設置周加權損益
開盤價=market['開盤價'].shift(-1)
最高價=market['最高價'].shift(-1)
最低價=market['最低價'].shift(-1)
收盤價=market['收盤價'].shift(-1)
index_open=(開盤價+最高價+最低價+收盤價)/4
index_open = index_open.to_frame(name='value')
index_open=resample_to_period(index_open, period=period)
monthreturn_market=((index_open['value'].shift(-1)/index_open['value'])-1).shift(2)#.loc[:'2025-01-17']

In [12]:
monthreturn

coid,1101,1102,1103,1104,1107,1108,1109,1110,1201,1203,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2010-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-22,-0.026662,-0.020878,-0.027416,-0.002053,0.018868,-3.390743e-02,-0.020501,-0.032258,-0.016959,0.118562,...,-0.001880,0.049625,0.033113,0.000811,-0.009524,0.031310,-0.011844,0.002901,-0.011372,-0.012879
2010-01-29,-0.075968,-0.066859,-0.053559,-0.051440,-0.060847,-6.991643e-02,-0.040698,-0.044792,-0.080206,-0.107732,...,-0.043785,-0.067070,-0.093407,-0.019457,-0.005495,-0.002999,-0.078706,-0.042430,-0.120782,-0.100537
2010-02-06,-0.056522,-0.052285,-0.060313,-0.060738,0.045070,-5.001497e-02,-0.023030,-0.041439,-0.100033,-0.044304,...,-0.026096,-0.051562,-0.029293,0.007441,-0.033149,-0.066165,-0.078491,-0.013092,-0.059311,-0.069966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,-0.013410,-0.021283,-0.032624,-0.000446,NaN,-1.111111e-02,-0.017808,-0.030393,-0.008345,-0.041867,...,-0.002497,-0.050276,0.002124,-0.032577,0.093489,0.018173,-0.012270,-0.022551,-0.013436,-0.004910
2025-01-03,-0.010485,-0.015101,0.001466,0.034344,NaN,2.220446e-16,-0.004881,0.007990,0.002805,0.004087,...,0.003129,0.010762,-0.040276,0.006122,-0.090076,0.008696,0.013753,-0.064888,-0.004864,-0.023026
2025-01-10,-0.004710,-0.005826,-0.009517,-0.020699,NaN,-4.321521e-03,-0.009811,-0.004268,-0.000699,-0.008766,...,0.007486,-0.011511,0.025953,0.014199,-0.033557,-0.035390,0.038950,0.010794,-0.010753,-0.005051


In [7]:
def calculate_beta(returns_oil, returns_stock):
    cov_matrix = np.cov(returns_oil, returns_stock)
    beta = cov_matrix[0, 1] / cov_matrix[0, 0]
    return beta

betas = monthreturn.loc['2015':].rolling(window=52).apply(lambda x: calculate_beta(monthreturn_market.loc['2015':].loc[x.index], x), raw=False)
betas = pd.concat([betas], axis=1)

KeyError: "[Timestamp('2025-01-20 00:00:00')] not in index"

In [52]:
barra所需資料=manager.tool_api_data("個股市值_元","本益比","股價淨值比","周轉率","股利殖利率")
barra所需資料=resample_to_period(barra所需資料, period=period)

資料集 '個股市值_元' 包含其他欄位（['Market_Cap_Dollars']），進行 pivot 操作。
資料集 '本益比' 包含其他欄位（['PER_TWSE']），進行 pivot 操作。
資料集 '股價淨值比' 包含其他欄位（['PBR_TWSE']），進行 pivot 操作。
資料集 '周轉率' 包含其他欄位（['Turnover']），進行 pivot 操作。
資料集 '股利殖利率' 包含其他欄位（['Dividend_Yield_TWSE']），進行 pivot 操作。


/var/folders/x2/75yr957x4dv95sfg2x05yf980000gn/T/ipykernel_21877/1128842498.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(columns=['datetime'])


In [55]:
###市值因子
cap=barra所需資料['Market_Cap_Dollars']

###市值自然對數
lncap=np.log(barra所需資料['Market_Cap_Dollars'])

###非線性市值
nonlinear_size =lncap ** 2

# 中市值
lower_quantile = cap.quantile(0.3, axis=1)
upper_quantile = cap.quantile(0.7, axis=1)
middle_cap = (cap.apply(lambda row: (row >= lower_quantile[row.name]) & (row <= upper_quantile[row.name]), axis=1).astype(int))

#動能因子
momentum = (monthreturn.shift(1).rolling(11).mean())

#短期反轉
short_term_reversal = (monthreturn.shift(1))

#季節因子
quarters = ((monthreturn.index.month - 1) // 3 + 1)
seasonality_factors = (monthreturn.groupby(quarters).transform('mean'))

#流通性
liquidity = barra所需資料['Turnover']

#帳面市值比
book_to_price = barra所需資料['PBR_TWSE']

###本益比因子
earning_to_price =  barra所需資料['PER_TWSE']

#股利殖利率
dividend_yield=barra所需資料['Dividend_Yield_TWSE']

In [56]:
def Zscore_Scaler_Capweight(df:pd.DataFrame, cap:pd.DataFrame) -> pd.DataFrame:
    capWeight = cap.div(cap.sum(axis=1), axis=0)
    return df.sub((df*capWeight).sum(axis=1), axis=0).div(df.std(axis=1), axis=0)

In [70]:
tse_filter_month=manager.tool_api_data("市場別")['Market']=='TWSE'
tse_filter_month=resample_to_period(tse_filter_month, period=period)
benchmark_weights = cap[tse_filter_month].div(cap[tse_filter_month].sum(axis=1),axis=0).fillna(0)

資料集 '市場別' 包含其他欄位（['Market']），進行 pivot 操作。


In [71]:
factor_exposure = pd.concat({'cap':Zscore_Scaler_Capweight(cap, cap[tse_filter_month]).fillna(0).stack(),
                             'lncap':Zscore_Scaler_Capweight(lncap, cap[tse_filter_month]).fillna(0).stack(),
                             'nonlinear_size':Zscore_Scaler_Capweight(nonlinear_size, cap[tse_filter_month]).fillna(0).stack(),
                             'middle_cap':Zscore_Scaler_Capweight(middle_cap, cap[tse_filter_month]).fillna(0).stack(),
                             'momentum':Zscore_Scaler_Capweight(momentum, cap[tse_filter_month]).fillna(0).stack(),
                             'short_term_reversal':Zscore_Scaler_Capweight(short_term_reversal, cap[tse_filter_month]).fillna(0).stack(),
                             'seasonality_factors':Zscore_Scaler_Capweight(seasonality_factors, cap[tse_filter_month]).fillna(0).stack(),
                             'liquidity':Zscore_Scaler_Capweight(liquidity, cap[tse_filter_month]).fillna(0).stack(),
                             'book_to_price':Zscore_Scaler_Capweight(book_to_price, cap[tse_filter_month]).fillna(0).stack(),
                             'earning_to_price':Zscore_Scaler_Capweight(earning_to_price, cap[tse_filter_month]).fillna(0).stack(),
                             'dividend_yield':Zscore_Scaler_Capweight(dividend_yield,cap[tse_filter_month]).fillna(0).stack(),
                             'betas':Zscore_Scaler_Capweight(betas, cap[tse_filter_month]).fillna(0).stack(),},axis=1)

In [99]:
industry=manager.tool_api_data("主產業別_英文")['Industry_Eng']
industry=resample_to_period(industry, period=period)

資料集 '主產業別_英文' 包含其他欄位（['Industry_Eng']），進行 pivot 操作。


In [101]:
industry_codes = industry.stack().unique().tolist()
industry_codes = [code for code in industry_codes if isinstance(code, str)]
encode_industries = {code: (industry == code).astype(int) for code in industry_codes}

In [78]:
def transform_to_industry_view(df):
    transformed_df = df.stack(level="Stock")  # 将 "Stock" 作为 index 的一部分
    transformed_df.columns.name = "Industry"  # 设置新列名为 "Industry"
    transformed_df.index.names = ["datetime", "Stock"]  # 修改索引名称 
    return transformed_df

In [108]:
result = encode_industries.copy()
ind_dict={}
for i in industry.stack().unique().tolist():
    ind = (result[i] * benchmark_weights)
    new_columns = pd.MultiIndex.from_product([[f'{i}'], ind.columns])
    ind_dict[f'{i}'] = pd.DataFrame(ind.values, index=ind.index, columns=new_columns)
ind_exposure= (pd.concat(ind_dict.values(), axis=1)).stack()

/var/folders/x2/75yr957x4dv95sfg2x05yf980000gn/T/ipykernel_21877/94173605.py:7: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  ind_exposure= (pd.concat(ind_dict.values(), axis=1)).stack()


# 儲存資料

In [110]:
factor_exposure.to_pickle("/Users/chieray/TMBA-ATD-coding/因子研究/factor_exposure.pkl")

In [111]:
ind_exposure.to_pickle("/Users/chieray/TMBA-ATD-coding/因子研究/ind_exposure.pkl")